In [1]:
import loralib as lora
from torch import nn
import torchvision
import torch
import torch.nn.functional as F

In [2]:
from train_on_NDI import *
from config import Config

cfg = Config()
device = torch.device(cfg.device)
set_all_seeds(cfg.seed)



In [3]:
def convert_layers_LoRA(model, convert_types=[nn.Linear, nn.Conv2d], rank_factor=10):
    named_modules = dict(model.named_modules())
    for name, module in find_modules(model=model, search_classes=convert_types):
        name_parts = name.split('.')
        if len(name_parts) > 1:
            parent_module = '.'.join(name_parts[:-1])
            parent = named_modules[parent_module]
            name = name.split('.')[-1]
        else:
            parent = model
        update_to_LoRA_layer(module, parent, name, rank_factor)
    return model


def update_to_LoRA_layer(module, parent, name, rank_factor=10):
    if isinstance(module, nn.Linear):
        setattr(parent, name, lora.Linear(module.in_features, module.out_features,
                                          r=min(module.in_features, module.out_features) // rank_factor))
    elif isinstance(module, nn.Conv2d):
        if type(module.kernel_size) is not int:
            kernel_size = module.kernel_size[0]
            assert all([dim == kernel_size for dim in module.kernel_size])
        else:
            kernel_size = module.kernel_size
        setattr(parent, name, lora.Conv2d(module.in_channels, module.out_channels, kernel_size,
                                          r=min(module.in_channels, module.out_channels) // 4, 
                                          stride=module.stride, padding=module.padding, 
                                          dilation=module.dilation, groups=module.groups))


def find_modules(model, search_classes=[nn.Linear, nn.Conv2d], exclusion=[lora.LoRALayer]):
    layers_to_replace = []
    for name, module in model.named_modules():
        for _class in exclusion:
            if isinstance(module, _class):
                continue
        for _class in search_classes:
            if isinstance(module, _class):
                layers_to_replace.append((name, module))
    return layers_to_replace

def load_checkpoints(base_encoder, ckpt_path):
    temp = torch.load(ckpt_path)['state_dict']
    state_dict = {}
    for k, v in temp.items():
        if 'encoder_q' in k:
            if 'fc' not in k:
                state_dict['.'.join(k.split('.')[1:])] = v
    base_encoder.load_state_dict(state_dict, strict=False)
    return base_encoder

def get_modified_resnet50():
    model = torchvision.models.resnet50()
    model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    origin_mlp_dim = model.fc.in_features
    model.fc = nn.Linear(origin_mlp_dim, 512)
    return model

model = get_modified_resnet50()
model = convert_layers_LoRA(model)
print(type(model.layer1[0].conv1))

<class 'loralib.layers.Conv2d'>


In [4]:
model = load_checkpoints(model, './checkpoints/ImageNet_ALL_CHECK_400_Epoch.pth')
lora.mark_only_lora_as_trainable(model)

In [5]:
logger_fname = datetime.datetime.now().strftime('%Y%m%d%H%M')
logger = get_logger(f'./logs/{logger_fname}.log')

logger.info(f'This training is to do {cfg.message_to_log}')

for i, images in enumerate(k_fold_train_validation_split(ORIGINAL_IMAGE, TARGET_IMAGE, 7)):
    wandb.init(project=cfg.message_to_log, group='lora_implementation', job_type='Imagenet_NDI_400E',
                name=f'fold {i}', config=cfg.__dict__)
    train_dataset = SingleChannelNDIDatasetContrastiveLearningWithAug(images, False, 200)
    val_dataset = SingleChannelNDIDatasetContrastiveLearningWithAug(images, True, 200)
    train_iter = DataLoader(train_dataset, cfg.batch_size, shuffle=True, drop_last=True)
    val_iter = DataLoader(val_dataset, batch_size=len(val_dataset))

    model = RetrievalModel(model)
    model = model.cuda()

    training_params = [param for param in model.parameters() if param.requires_grad]
    optimizer = torch.optim.SGD(params=training_params, lr=cfg.base_lr, weight_decay=cfg.weight_decay, momentum=cfg.momentum)
    criterion = nn.CrossEntropyLoss()

    # scheduler = None
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=cfg.epochs, eta_min=cfg.base_lr * 0.01)

    train(cfg, logger, train_iter, val_iter, model, criterion, optimizer, cfg.epochs, scheduler=scheduler,
            save_folder=cfg.output_dir, wandb_config=True, device=device)
    wandb.finish()




[2023-04-27 16:39:33,712][2436042988.py][line:4][INFO] This training is to do Lora test
[2023-04-27 16:39:34,336][jupyter.py][line:224][ERROR] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: zombiesama18. Use `wandb login --relogin` to force relogin


/home/gp.sc.cc.tohoku.ac.jp/duanct/miniconda3/envs/py2cuda118/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Training Progress	Epoch: [1/50][0/5]	Batch Time  7.655 ( 7.655)	Data Time  2.187 ( 2.187)	Train Loss 1.0622e+01 (1.0622e+01)


[2023-04-27 16:40:03,323][train_on_NDI.py][line:206][INFO] Epoch: [0/50], train loss 9.827414512634277, val loss 6.525238990783691, val acc @ 10 8.333333015441895, val acc @ 20 12.5, val acc @ 30 16.66666603088379lr [0.005]


Validation Progress	Epoch: [1/50][0/1]	Val Loss 6.5252e+00 (6.5252e+00)	Val Acc@10   8.33 (  8.33)	Val Acc@20  12.50 ( 12.50)	Val Acc@30  16.67 ( 16.67)
Training Progress	Epoch: [2/50][0/5]	Batch Time  2.452 ( 2.452)	Data Time  2.405 ( 2.405)	Train Loss 1.0296e+01 (1.0296e+01)


...

[2023-04-27 16:47:07,554][train_on_NDI.py][line:206][INFO] Epoch: [49/50], train loss 8.804769325256348, val loss 7.867000579833984, val acc @ 10 4.166666507720947, val acc @ 20 4.166666507720947, val acc @ 30 12.5lr [5.488384714002789e-05]
[2023-04-27 16:47:07,555][train_on_NDI.py][line:230][INFO] Training Finished!


Validation Progress	Epoch: [50/50][0/1]	Val Loss 7.8670e+00 (7.8670e+00)	Val Acc@10   4.17 (  4.17)	Val Acc@20   4.17 (  4.17)	Val Acc@30  12.50 ( 12.50)


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/train loss,█▆▆▇█▆▄▄▄▅▅▅▄▄▄▅▄▄▄▄▂▄▄▄▂▃▃▂▁▂▃▃▁▂▁▂▃▁▂▂
val/val acc @ 10,▆▁▆██▆▁▁▁▆▃▆▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃
val/val acc @ 20,▅▁▇█▇▄▂▁▁▄▅▇▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
val/val acc @ 30,▅▃▆█▇▃▂▁▁▃█▇▅▃▄▄▄▅▄▄▄▃▃▃▄▄▃▄▃▃▃▃▄▃▄▃▃▃▄▄
val/val loss,▁▁▆▇▅▄▃▂▃▃▃▅▇▇████████████▇▇███▇▇▇▇███▇▇
epoch,50
train/train loss,8.80477
val/val acc @ 10,4.16667
val/val acc @ 20,4.16667
val/val acc @ 30,12.5


/home/gp.sc.cc.tohoku.ac.jp/duanct/miniconda3/envs/py2cuda118/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


TypeError: forward() missing 1 required positional argument: 'im_k'